In [69]:
import pandas as pd
import pickle
import numpy as np
from S2_model import rag_model 
from s3_model import agent_model 

In [70]:
params = {}

params['Nreplicates'] = 5
params['llm_choice'] = "ChatGPT"
params['model'] = None #llama3.1" #model choice if using a local Ollama model

system_prompt = """You are a Foundational Nature AI capable of informing questions about biodiversity and conservation relevant for real-world decisions. 
                    Please respond with a score between 0 and 1, where 1 indicates that you think the species is very likely to be present there and 0 
                    indicates the species is highly unlikely to be found there. Please ensure your response is in the format of the score followed by a
                    comma and then the justification."""

In [7]:
s2_model = rag_model(dossier_path='../data/retrieval_dossier/wikipedia-en-dwca-species-descriptions.csv', system_prompt= system_prompt,
               llm_choice = params['llm_choice'], model = params['model'], persist_directory='../training/wikipedia')

131472


In [19]:
s3_model = agent_model(system_prompt = system_prompt,llm_choice = params['llm_choice'], model = params['model'])

In [8]:
question = 'Can you tell me how likely I am to find the snowy owl (Bubo scandiacus) in my garden in Comberton, Cambridgeshire'
response = s2_model.invoke_response(question)



In [20]:
response_s3 = s3_model.invoke_response(question)



> Entering new AgentExecutor chain...

Invoking: `inaturalist` with `{'taxon': 'Bubo scandiacus'}`


[{'name': 'United States', 'latitude': '45.9643322098', 'longitude': '-113.2690112346', 'place_type_name': 'Country'}, {'name': 'Massachusetts', 'latitude': '42.1600897179', 'longitude': '-71.503987243', 'place_type_name': 'State'}, {'name': 'Nebraska', 'latitude': '41.5271511252', 'longitude': '-99.8108558603', 'place_type_name': 'State'}, {'name': 'Delaware', 'latitude': '38.9949683916', 'longitude': '-75.4524914612', 'place_type_name': 'State'}, {'name': 'District of Columbia', 'latitude': '38.9047738879', 'longitude': '-77.0162909031', 'place_type_name': 'State'}, {'name': 'Alaska', 'latitude': '63.7429890154', 'longitude': '-179.6857025', 'place_type_name': 'State'}, {'name': 'Virginia', 'latitude': '37.5108673459', 'longitude': '-78.6663604926', 'place_type_name': 'State'}, {'name': 'Rhode Island', 'latitude': '41.5946290792', 'longitude': '-71.5247424878', 'place_type_name': 'S

In [22]:
def filter_response(response):
    if {'answer'} <= response.keys():
        filtered_response = response['answer']
    elif {'output'} <= response.keys():
        filtered_response = response['output']

    return filtered_response

In [23]:
filter_response(response)

'0, Snowy owls are native to Arctic regions and typically breed in the northern circumpolar areas. They are highly unlikely to be found in Cambridgeshire, which is far south of their usual range. While they are nomadic and can sometimes be found at more southerly latitudes, it is extremely rare for them to appear in areas like Comberton.'

In [24]:
filter_response(response_s3)

'0, The snowy owl (Bubo scandiacus) is primarily found in the United States, particularly in states like Massachusetts, Nebraska, Delaware, and Alaska. It is not native to the UK, and especially not to regions like Cambridgeshire. Thus, it is highly unlikely to encounter a snowy owl in your garden in Comberton, Cambridgeshire.'

In [5]:
with open(f'../output/Q0_s1.txt', 'r') as f:
    print(pd.read_table(f, skiprows=0))
    # for line in f:
    #     line.to_dict
    for index, row in pd.read_table(f, skiprows=0).iterrows():
        response = row
        print(response)

  {'answer': '0, Darwin’s Fox (Lycalopex fulvipes) is endemic to southern Chile and is not found outside this region. Parque Nacional Bahuaja Sonene is located in Peru, far outside the natural range of this species, 1'}
0  {'answer': '0, Darwin’s Fox (Lycalopex fulvipe...                                                                                                                                                                       
1  {'answer': '0, Darwin’s Fox (Lycalopex fulvipe...                                                                                                                                                                       
2  {'answer': '0, Darwin’s Fox (Lycalopex fulvipe...                                                                                                                                                                       
3  {'answer': '0, Darwin’s Fox (Lycalopex fulvipe...                                                                    

EmptyDataError: No columns to parse from file

In [71]:
import pickle

params['pkl_out'] = f"output/{params['llm_choice']}_All_Model_Q_responses.pkl"

In [72]:
in_file = open('../'+params['pkl_out'], 'rb')
responses = pickle.load(in_file)
in_file.close()

In [74]:
def quantitative_species_presence_metric(responses,i):
    df = read_and_process_species_responses(responses)
    diffs = df['Value'] - data['Value'][i]
    mean_diff = np.mean(diffs)
    return df['Value'], diffs, mean_diff



def read_and_process_species_responses(responses):
    """
    Reads a text file and splits each line into a numeric value and a text description.
    Assumes each line starts with a numeric value followed by a comma, then the text.

    Parameters:
    responses (dict): dictionary of .
    
    Returns:
    DataFrame: A pandas DataFrame with two columns: 'Numeric' and 'Text'.
    """
    data = []  # List to store the split data

    # Open the file and process each line
    #with open(file_path, 'r', encoding='utf-8') as file:
    for r in responses:
        # Strip whitespace and split the line at the first comma
        parts = r.strip().split(',', 1)
        #Check that there are 2 parts and that it doesn't start with "Note:"
        if len(parts) == 2 and not parts[0].startswith("Note:"):
            numeric_value = float(parts[0].strip())  # Convert the numeric part to float
            text_description = parts[1].strip()
            data.append([numeric_value, text_description])

    # Convert the list of data into a DataFrame
    df = pd.DataFrame(data, columns=['Value', 'Justification'])
    return df


In [75]:
def read_query_file_and_construct_questions(file_path):
    """
    Reads a custom-formatted file where the first line contains a question with placeholders,
    and the subsequent lines contain data in CSV format. This function replaces the placeholders
    in the question with data from each row.

    Parameters:
    file_path (str): The path to the file to be read.
    
    Returns:
    list: A list of questions with data inserted from each row.
    """
    with open(file_path, 'r') as file:
        # Extract the first line and get the question template
        first_line = file.readline().strip()
        if first_line.startswith('#System_prompt: '):
            system_prompt = first_line[len('"#System_prompt:'):].strip()
        else:
            raise ValueError("The file does not start with a proper system prompt header.")
        
        second_line = file.readline().strip()
        if second_line.startswith('#Prompt: '):
            question_template = second_line[len('#Prompt: '):].strip()
        else:
            raise ValueError("The file does not start with a proper question header.")

        # Read the CSV data that follows
        data = pd.read_csv(file, skiprows=0)  # We already read the first and second lines, so the cursor is at the second line

    # print(repr(question_template))
    # q_data_columns = re.findall(r'\{([^}]+)\}',question_template)
    # print(q_data_columns)


    # List to hold all questions filled with data
    filled_questions = []

    # Fill the question template with data from each row
    for index, row in data.iterrows():
        filled_question = question_template.format(**row.to_dict())
        filled_questions.append(filled_question)
    
    return system_prompt,filled_questions, data

def filter_response(response):
    if {'answer'} <= response.keys():
        filtered_response = response['answer']
    elif {'output'} <= response.keys():
        filtered_response = response['output']

    return filtered_response

In [76]:
#Species locations
file_path = '../eval/species_geospatial_queries_responses - species_presences.csv'  # Replace this with the path to your file
try:
    system_prompt, queries, data = read_query_file_and_construct_questions(file_path)
    print(system_prompt)

except Exception as e:
    print("Error reading queries:", e)

You are a Foundational Nature AI capable of informing questions about biodiversity and conservation relevant for real-world decisions. Please respond with a score between 0 and 1, where 1 indicates that you think the species is very likely to be present there and 0 indicates the species is highly unlikely to be found there. Please provide your response in the format of the score then a comma, then the justification for the score, then a comma and then a measure of your confidence, again scored from 0 to 1 (0 is very low confidence, 1 is very confident in your answer).


In [97]:
params['pkl_out'] = f"../output/{params['llm_choice']}_All_Model_Q_responses.pkl"
models = ['s1','s2','s3']
diffs = {}
vals = {}
mean_diff = {}
filtered_responses = {}
responses = {}

if not responses:
    print(f'Loading responses from: {params['pkl_out']}')
    in_file = open(params['pkl_out'], 'rb')
    responses = pickle.load(in_file)
    in_file.close()


Loading responses from: ../output/ChatGPT_All_Model_Q_responses.pkl


In [96]:
responses['0']['s2']

[{'input': 'Can you tell me if Darwin’s Fox (Lycalopex fulvipes) can be found in Parque Nacional Bahuaja Sonene, Peru',
  'context': [Document(metadata={'source': 'data/retrieval_dossier/wikipedia-en-dwca-species-descriptions.csv'}, page_content='16220,"Darwin\'s fox. Lycalopex fulvipes. Ecology. Darwin\'s fox is generally believed to be a forest obligate species found only in southern temperate rainforests. They only occur in areas of primary forest on Chiloé and on the mainland. They are most active at twilight and before sunrise. In contrast to other Lycalopex species, Darwin\'s fox prefers open spaces. The population of Chiloé has about 200 individuals, and Nahuelbuta on the mainland contains about 50 individuals. The total population size is about 250 mature individuals with at least 90% of the population occurring in one subspopulation (Chiloé Island). Although the species is protected in Nahuelbuta National Park, substantial mortality sources exist when foxes move to lower, unpr

In [99]:
i = 2
model= 's1'
[filter_response(r) for r in responses[str(i)][model]]

['0',
 'The Fosa (Cryptoprocta ferox) is endemic to Madagascar and is not found outside of the island. Therefore, it is highly unlikely to be found in Parque Nacional de Gorongosa, Mozambique.',
 'The Fosa (Cryptoprocta ferox) is endemic to Madagascar and is not found in Parque Nacional de Gorongosa, Mozambique.',
 'The Fosa (Cryptoprocta ferox) is endemic to Madagascar and is not found in Parque Nacional de Gorongosa, Mozambique.',
 'The Fosa (Cryptoprocta ferox) is endemic to Madagascar and is not found in Mozambique, including Parque Nacional de Gorongosa.',
 'The Fosa (Cryptoprocta ferox) is endemic to Madagascar and is not found in Mozambique, including Parque Nacional de Gorongosa.',
 'The Fosa (Cryptoprocta ferox) is an endemic species to Madagascar, and it is highly unlikely to be found in Parque Nacional de Gorongosa, Mozambique.',
 'The fosa (Cryptoprocta ferox) is endemic to Madagascar and is not found in Mozambique, including Parque Nacional de Gorongosa.',
 'The Fosa (Cryp

In [77]:


for i, question in enumerate(queries):
    
    diffs[str(i)] = {}
    mean_diff[str(i)] = {}
    vals[str(i)] = {}
    for model in models:
        filtered_responses = [filter_response(r) for r in responses[str(i)][model]]
 
        vals[str(i)]['vals_'+model],diffs[str(i)]['diffs_'+model], mean_diff[str(i)][model] = quantitative_species_presence_metric(filtered_responses,i)
        


Loading responses from: ../output/ChatGPT_All_Model_Q_responses.pkl


ValueError: could not convert string to float: 'The fossa (Cryptoprocta ferox) is endemic to Madagascar and not native to mainland Africa'

In [67]:
k = '5'
pdf_details = pd.concat([pd.DataFrame(vals[k]),pd.DataFrame(diffs[k])], axis = 1)

np_diffs = np.array(pd.DataFrame(diffs[k]))
print(np_diffs)
rmse = np.sqrt(np.mean(np.square(np_diffs),axis = 0))
print(f"rmse = {rmse}. mean = {np.mean(np_diffs,axis = 0)}")

[[-0.05 -0.05 -0.1 ]
 [-0.05 -0.15 -0.15]
 [-0.15 -0.15 -0.15]
 [-0.05 -0.15 -0.15]
 [-0.05 -0.15 -0.15]]
rmse = [0.08062258 0.13601471 0.14142136]. mean = [-0.07 -0.13 -0.14]


In [53]:
np_diffs

array({'diffs_s1': 0   -0.05
1   -0.05
2   -0.15
3   -0.05
4    0.05
Name: Value, dtype: float64, 'diffs_s2': 0    0.05
1    0.05
2    0.05
3    0.05
4    0.05
Name: Value, dtype: float64, 'diffs_s3': 0    0.00
1   -0.10
2   -0.15
3   -0.15
4    0.00
Name: Value, dtype: float64}, dtype=object)